In this section the API selected will be used so that we load our ES db

we ll use docker for elastic search and fill the details in the .ENV file

- NYT-Times NewsWire API
- NYT-Books API
- GoodReads

// end points
// list of book categories : https://api.nytimes.com/svc/books/v3/lists/names.json
// number of reviewed book : https://api.nytimes.com/svc/books/v3/lists/best-sellers/history.json
// good reads : https://www.goodreads.com/book/review_counts.json
//

// need to establish the following :
//1. connection to the ES container
//
//2. loading the api request from the two api
//2. 1 for each API define loops and requirements


1. Connection to the container // Windows installation

1.Install Docker: Download and install Docker Desktop for Windows from the official Docker website. Follow the installation instructions specific to your Windows version.
2.Launch Docker: Once Docker is installed, launch Docker Desktop from the start menu or desktop shortcut. Make sure it's running before proceeding to the next steps.
3.Open a Command Prompt: Open the Command Prompt or PowerShell on your Windows machine.
4.Pull the Elasticsearch Docker Image: Use the following command to pull the official Elasticsearch Docker image from Docker Hub:

bash
Copy code
docker pull docker.elastic.co/elasticsearch/elasticsearch:8.8.1

5.Create a Docker Container
docker run -d --name elasticsearch_container -p 9200:9200 -p 9300:9300 -e "discovery.type=single-node" docker.elastic.co/elasticsearch/elasticsearch:8.8.1
docker run -d --name elasticsearch  -v /data/dump/:/usr/share/elasticsearch/data -p 9400:9400 -p 9500:9200 -e "discovery.type=single-node" elasticsearch:8.8.1
docker run -d --name elasticsearch761v2  -v /data/dump/:/usr/share/elasticsearch/data -p 9400:9400 -p 9500:9200 -e "discovery.type=single-node" elasticsearch:7.6.1

docker run --name elasticSearch -p 9200:9200 -p 9300:9300 -e "discovery.type=single-node" docker.elastic.co/elasticsearch/elasticsearch:7.5.2

6.Verify Elasticsearch Installation: Open your web browser and navigate to http://localhost:9200.


Once the connection is established we want to analyse in details the API and load them one by one without overloading the API from NYT or GoodReads

To interact with ES firstly you need to : pip install elasticsearch


In [22]:
from elasticsearch import Elasticsearch
from elasticsearch.helpers import bulk
from dotenv import load_dotenv


es = Elasticsearch(hosts = "http://@localhost:9200")


index_name = 'nyt-newswire'
newswire_mapping = {
        'properties': {
            'abstract': {'type': 'text', 'analyzer': 'english'},
            'byline': {'type': 'text', 'analyzer': 'english'},
            'created_date': {'type': 'date'},
            'des_facet': {'type': 'keyword'},
            'first_published_date': {'type': 'date'},
            'geo_facet': {'type': 'keyword'},
            'item_type': {'type': 'keyword'},
            'kicker': {'type': 'text'},
            'material_type_facet': {'type': 'keyword'},
            'multimedia': {
                'type': 'nested',
                'properties': {
                    'caption': {'type': 'text', 'analyzer': 'english'},
                    'copyright': {'type': 'text', 'analyzer': 'english'},
                    'format': {'type': 'keyword'},
                    'height': {'type': 'integer'},
                    'subtype': {'type': 'keyword'},
                    'type': {'type': 'keyword'},
                    'url': {'type': 'text', 'analyzer': 'english'},
                    'width': {'type': 'integer'}
                }
            },
            'org_facet': {'type': 'keyword'},
            'per_facet': {'type': 'keyword'},
            'published_date': {'type': 'date'},
            'section': {'type': 'keyword'},
            'slug_name': {'type': 'keyword'},
            'source': {'type': 'text', 'analyzer': 'english'},
            'subheadline': {'type': 'text', 'analyzer': 'english'},
            'subsection': {'type': 'keyword'},
            'thumbnail_standard': {'type': 'text', 'analyzer': 'english'},
            'title': {'type': 'text'},
            'updated_date': {'type': 'date'},
            'uri': {'type': 'keyword'},
            'url': {'type': 'text', 'analyzer': 'english'}
        }
    }

newswire_settings = {
    "number_of_shards": 2,
    "number_of_replicas": 2
}

 # Create the index
response = es.indices.create(index=index_name, mappings=newswire_mapping, 
                             settings=newswire_settings)

# Check the response
  
if response['acknowledged']:
    print('Index created successfully.')
else:
    print('Failed to create index.')

Index created successfully.


In [23]:
from elasticsearch import Elasticsearch
from elasticsearch.helpers import bulk
from dotenv import load_dotenv

##########################################################################
# we prepare the mapping for the new ESindex
##########################################################################
##########################################################################
# Mapping and index for yhr NYT - Book API -- best sellers
##########################################################################
#To be fixed 


es = Elasticsearch(hosts = "http://@localhost:9200")

index_name = 'nyt-books'
BS_books_index_body = {
    'properties': {
    'title': {'type': 'text', 'analyzer': 'english'},
    'description': {'type': 'text', 'analyzer': 'english'},
    'contributor': {'type': 'text', 'analyzer': 'english'},
    'author': {'type': 'text', 'analyzer': 'english'},
    'contributor_note': {'type': 'text', 'analyzer': 'english'},
    'price': {'type': 'float'},
    'age_group': {'type': 'keyword'},
    'publisher': {'type': 'text', 'analyzer': 'english'},
    'isbns': {
        'type': 'nested',
        'properties': {
            'isbn10': {'type': 'keyword'},
            'isbn13': {'type': 'keyword'},
        }
    },
    'ranks_history': {
        'type': 'nested',
        'properties': {
            'primary_isbn10': {'type': 'keyword'},
            'primary_isbn13': {'type': 'keyword'},
            'rank': {'type': 'integer'},
            'list_name': {'type': 'text', 'analyzer': 'english'},
            'display_name': {'type': 'text', 'analyzer': 'english'},
            'published_date': {'type': 'date'},
            'bestsellers_date': {'type': 'date'},
            'weeks_on_list': {'type': 'integer'},
            'ranks_last_week': {'type': 'integer', 'null_value': None},
            'asterisk': {'type': 'integer'},
            'dagger': {'type': 'integer'},
        }
    },
    'reviews': {
        'type': 'nested',
        'properties': {
            'book_review_link': {'type': 'keyword'},
            'first_chapter_link': {'type': 'keyword'},
            'sunday_review_link': {'type': 'keyword'},
            'article_chapter_link': {'type': 'keyword'},
            }
        },
    }
}

books_settings = {
    "number_of_shards": 2,
    "number_of_replicas": 2
}


 # Create the index
response = es.indices.create(index=index_name, mappings=BS_books_index_body, 
                             settings=books_settings)

# Check the response
  
if response['acknowledged']:
    print('Index created successfully.')
else:
    print('Failed to create index.')

Index created successfully.


In [31]:
from elasticsearch import Elasticsearch
from elasticsearch.helpers import bulk
from dotenv import load_dotenv

##########################################################################
# we prepare the mapping for the new ESindex
##########################################################################
##########################################################################
# Mapping and index for yhr NYT - MOVIES API --
##########################################################################

es = Elasticsearch(hosts="http://@localhost:9200")

index_name = 'nyt-movies'
BS_movies_index_body = {
    "properties": {
        "byline":  {'type': 'text', 'analyzer': 'english'},
        "critics_pick": {"type": "integer"},
        "date_updated": {"type": "date", "format": "yyyy-MM-dd HH:mm:ss"},
        "display_title": {'type': 'text', 'analyzer': 'english'},
        "headline":  {'type': 'text', 'analyzer': 'english'},
        "link": {
            "properties": {
                "suggested_link_text":  {'type': 'text', 'analyzer': 'english'},
                "type": {"type": "keyword"},
                "url": {"type": "text"}
            }
        },
        "mpaa_rating": {"type": "keyword"},
        "multimedia": {
            "properties": {
                "height": {"type": "integer"},
                "src":  {'type': 'text', 'analyzer': 'english'},
                "type": {"type": "keyword"},
                "width": {"type": "integer"}
            }
        },
        "opening_date": {"type": "date", "format": "yyyy-MM-dd"},
        "publication_date": {"type": "date", "format": "yyyy-MM-dd"},
        "summary_short": {"type": "text"}
    }
}

movies_settings = {
    "number_of_shards": 2,
    "number_of_replicas": 2
}

# Create the index
response = es.indices.create(index=index_name, mappings=BS_movies_index_body,
                             settings=movies_settings)

# Check the response

if response['acknowledged']:
    print('Index created successfully.')
else:
    print('Failed to create index.')

Index created successfully.


The following are the requests and loading in the Elastic search DB


In [30]:

from elasticsearch import Elasticsearch
from elasticsearch.helpers import bulk
from dotenv import load_dotenv
import os
import pprint
import requests
import datetime
import time
import json
pp = pprint.PrettyPrinter(indent=4)
load_dotenv()
api_key = os.getenv("API_KEY")

########################################################
########################################################
# NEWSWIRE API
########################################################
########################################################


#Define the variables used in the API
#Get the list of sections needed to call the API Newswire
reqSections =  requests.get(f'https://api.nytimes.com/svc/news/v3/content/section-list.json?&api-key={api_key}')
sectionsList= [item['section'] for item in reqSections.json()['results']]
print(sectionsList)


########################################################
########################################################
# for tests ONLY ###
########################################################
sectionsList = sectionsList[:2]
print(sectionsList)
########################################################
########################################################

# Create a connection to Elasticsearch  
es = Elasticsearch(hosts="http://@localhost:9200")  
# Proper index on the contenerize ES Db 
index_name = 'nyt-newswire'

# iterate through the list 
for section in sectionsList:
     
    #Request the Api
    content = requests.get(f'https://api.nytimes.com/svc/news/v3/content/all/{section}.json?&api-key={api_key}')
    #save into the ES DB
    res = content.json()
    pp.pprint(res) 
    docs = res['results']

    # Prepare the documents for bulk indexing
    actions = []
    for doc in docs:
        action = {
            "_index": index_name,
            "_source": doc
        }
        actions.append(action)

    now = datetime.datetime.now()
    print(f'query index:{section} at :{now}' )

    # Perform the bulk indexing
    response = bulk(es, actions)

    # Check the response
    if not response[1]:
        print(f'{section} saved successfully')
    else:
        print('Failed to save content.')
    

    ######################################################
    time.sleep(13) ##### TO MODIFY ACCORDING API ALLOWANCE
    #################### 5 requests max per minute ######
    ######################################################
    

['admin', 'arts', 'automobiles', 'books', 'briefing', 'business', 'climate', 'corrections', 'crosswords & games', 'education', 'en español', 'fashion', 'food', 'guides', 'health', 'home & garden', 'home page', 'job market', 'lens', 'magazine', 'movies', 'multimedia/photos', 'new york', 'obituaries', 'opinion', 'parenting', 'podcasts', 'reader center', 'real estate', 'science', 'smarter living', 'sports', 'style', 'sunday review', 't brand', 't magazine', 'technology', 'the learning network', 'the upshot', 'the weekly', 'theater', 'times insider', 'today’s paper', 'travel', 'u.s.', 'universal', 'video', 'well', 'world', 'your money']
['admin', 'arts']
{   'copyright': 'Copyright (c) 2023 The New York Times Company.  All Rights '
                 'Reserved.',
    'num_results': 500,
    'results': [   {   'abstract': '',
                       'byline': '',
                       'created_date': '2023-05-01T10:12:43-04:00',
                       'des_facet': None,
                      

In [12]:
########################################################
########################################################
# NEWSWIRE API -TEST ALL REQUEST
# The API sends back 500 articles meta for each section
# It is not possible to get more than 25000 articles
########################################################
########################################################

from elasticsearch import Elasticsearch
from pprint import pprint

# Create a connection to Elasticsearch
es = Elasticsearch(hosts="http://localhost:9200")
index_name = 'nyt-newswire'

# Define the search query
search_query = {
    "query": {
        "match_all": {}
    }
}

# Perform the search request
response = es.search(index=index_name, body=search_query)

# Process and print the search results
pprint(response['hits']['hits'])

C:\Users\ep211.000\AppData\Local\Temp\ipykernel_13360\2976581009.py:24: DeprecationWarning: The 'body' parameter is deprecated and will be removed in a future version. Instead use individual parameters.
  response = es.search(index=index_name, body=search_query)


[{'_id': 'VERQ4ogBhcLL25HdKowd',
  '_index': 'nyt-newswire',
  '_score': 1.0,
  '_source': {'abstract': '',
              'byline': '',
              'created_date': '2023-05-01T10:12:43-04:00',
              'des_facet': None,
              'first_published_date': '2023-05-01T10:12:43-04:00',
              'geo_facet': None,
              'item_type': 'Video',
              'kicker': '',
              'material_type_facet': 'Video',
              'multimedia': [{'caption': '',
                              'copyright': 'Storyful',
                              'format': 'Standard Thumbnail',
                              'height': 75,
                              'subtype': 'photo',
                              'type': 'image',
                              'url': 'https://static01.nyt.com/images/2023/05/01/01vid-ukraine-russia-carousel-13028-cover/01vid-ukraine-russia-carousel-13028-cover-thumbStandard.jpg',
                              'width': 75},
                             {

In [29]:
########################################################
########################################################
# NYT BOOKS API endpoint: best-sellers/history.json
# The API sends back 20 for each query 
# It is possible to get over 35K books review
########################################################
########################################################


from elasticsearch import Elasticsearch
from elasticsearch.helpers import bulk
from dotenv import load_dotenv
import os
import pprint
import requests
import datetime
import time
import json
pp = pprint.PrettyPrinter(indent=4)
load_dotenv()
api_key = os.getenv("API_KEY")



#connect to ES container in local 
es = Elasticsearch(hosts = "http://@localhost:9200")
index_name = 'nyt-books'

######################################################################################
# CHANGE THESE VALUES IF YOU NEED TO RESTART LOADING
# OR WANT TO TEST THE PROCESS 
######################################################################################
offsetvalue=0;    ###### IMPORTANT- OFFSETTING VALUE FOR THE START  OF THE WHILE LOOP
maxAPICall= 2     ##### To modify up to 500 if you want to load up to your daily limit
######################################################################################

offsetFactor=20 
endpointHits = 35311
continueLoading= True
APICallDailyIndex = 0  ##### MAX VAL  500


# iterate through the list 
while(continueLoading):
     
    APICallDailyIndex +=1
    continueLoading = APICallDailyIndex<maxAPICall

    #Request the Api
    content = requests.get(f"https://api.nytimes.com/svc/books/v3/lists/best-sellers/history.json?offset={offsetvalue}&api-key={api_key}")
    
    #save into the ES DB
    res = content.json()
    pp.pprint(res) 
    docs = res['results']
    
   # Prepare the documents for bulk indexing
    actions = []
    for doc in docs:
        action = {
            "_index": index_name,
            "_source": doc
        }
        actions.append(action)


    now = datetime.datetime.now()
    print(f'query index:{APICallDailyIndex} at :{now}' )

    # Perform the bulk indexing
    response = bulk(es, actions)

    # Check the response
    if not response[1]:
        print(f'{APICallDailyIndex*offsetFactor}`/35311 books saved successfully')
    else:
        print('Failed to save content.')

  
    ######################################################
    time.sleep(12) ##### TO MODIFY ACCORDING API ALLOWANCE
    ######################################################

print('end of the API loop')


{   'copyright': 'Copyright (c) 2023 The New York Times Company.  All Rights '
                 'Reserved.',
    'num_results': 35320,
    'results': [   {   'age_group': '',
                       'author': 'Diana Gabaldon',
                       'contributor': 'by Diana Gabaldon',
                       'contributor_note': '',
                       'description': 'The author of the Outlander novels '
                                      'gives tips on writing sex scenes, '
                                      'drawing on examples from the books.',
                       'isbns': [   {   'isbn10': '0399178570',
                                        'isbn13': '9780399178573'}],
                       'price': '0.00',
                       'publisher': 'Dell',
                       'ranks_history': [   {   'asterisk': 0,
                                                'bestsellers_date': '2016-08-20',
                                                'dagger': 0,
                 

In [20]:
########################################################
########################################################
# NYT BOOKS API endpoint: best-sellers/history 
#-TEST ALL REQUEST
# The API sends back 500 articles meta for each section
# It is not possible to get more than 25000 articles
########################################################
########################################################

from elasticsearch import Elasticsearch
from pprint import pprint

# Create a connection to Elasticsearch  
#connect to ES container in local 
es = Elasticsearch(hosts = "http://@localhost:9200")
index_name = 'nyt-books'


# Define the search query
search_query = {
    "query": {
        "match_all": {}
    }
}

# Perform the search request
response = es.search(index=index_name, body=search_query)

# Process and print the search results
pprint(response['hits']['hits'])

C:\Users\ep211.000\AppData\Local\Temp\ipykernel_13360\2880708551.py:27: DeprecationWarning: The 'body' parameter is deprecated and will be removed in a future version. Instead use individual parameters.
  response = es.search(index=index_name, body=search_query)


[{'_id': 'bERd4ogBhcLL25HdM40c',
  '_index': 'nyt-books',
  '_score': 1.0,
  '_source': {'age_group': '',
              'author': 'Diana Gabaldon',
              'contributor': 'by Diana Gabaldon',
              'contributor_note': '',
              'description': 'The author of the Outlander novels gives tips on '
                             'writing sex scenes, drawing on examples from the '
                             'books.',
              'isbns': [{'isbn10': '0399178570', 'isbn13': '9780399178573'}],
              'price': '0.00',
              'publisher': 'Dell',
              'ranks_history': [{'asterisk': 0,
                                 'bestsellers_date': '2016-08-20',
                                 'dagger': 0,
                                 'display_name': 'Advice, How-To & '
                                                 'Miscellaneous',
                                 'list_name': 'Advice How-To and Miscellaneous',
                                 'primary_

In [36]:
########################################################
########################################################
# NYT - MOVIE API 
# The API sends back 20 for each query 
# It is possible to get over 31300K books review
########################################################
########################################################

from elasticsearch import Elasticsearch
from elasticsearch.helpers import bulk
from dotenv import load_dotenv
import os
import pprint
import requests
import datetime
import time
pp = pprint.PrettyPrinter(indent=4)
load_dotenv()
api_key = os.getenv("API_KEY")



# Create a connection to Elasticsearch  
#connect to ES container in local 
index_name = 'nyt-movies'
es = Elasticsearch(hosts = "http://@localhost:9200")


########################################################
maxQueries= 6 ##### TO DEFINE !!!! According to API daily limit
########################################################
queryIndex=0     
offsetFactor=20

#API ARGUMENTS#############################################
type = "all"
offsetValue=0 ##  Max value tested == 31300 

# iterate through the list 
while(queryIndex<maxQueries): 

   

    # Create a connection to Elasticsearch  
    #Request the ApI
    print(f'query index:{offsetValue} at :{now}' )
    content = requests.get(f'https://api.nytimes.com/svc/movies/v2/reviews/{type}.json?offset={offsetValue}&api-key={api_key}')
    #save into the ES DB
    res = content.json()
    pp.pprint(res) 
    docs = res['results']

    queryIndex +=1
# Prepare the documents for bulk indexing
    ### this code could be made as a function but for
    # the sake of the cell singularity we leave it that way 
    actions = []
    for doc in docs:
        action = {
            "_index": index_name,
            "_source": doc
        }
        actions.append(action)

    # Perform the bulk indexing
    response = bulk(es, actions)

    # Check the response
    if not response[1]:
        print('saved successfully')
    else:
        print('Failed to save content.')

    now = datetime.datetime.now()
    print(f'query index:{queryIndex} at :{now}' )
  ################################################################
    #Define the logic to increment in a writer has more than 20 entries
 
    # has more pages 
    hasMore = res['has_more']
    print(f'hasmore:{hasMore} at :{now}' )

    if hasMore:
        offsetValue += offsetFactor
    else:     
        offsetValue=0 
    ################################################################


    ######################################################
    time.sleep(13) ##### TO MODIFY ACCORDING API ALLOWANCE
    ######################################################

query index:0 at :2023-06-22 12:53:14.867883
{   'copyright': 'Copyright (c) 2023 The New York Times Company. All Rights '
                 'Reserved.',
    'has_more': True,
    'num_results': 20,
    'results': [   {   'byline': 'Manohla Dargis',
                       'critics_pick': 0,
                       'date_updated': '2023-06-22 09:01:17',
                       'display_title': 'Revoir Paris',
                       'headline': '‘Revoir Paris’ Review: Recovering '
                                   'Fragments of Memory',
                       'link': {   'suggested_link_text': 'Read the New York '
                                                          'Times Review of '
                                                          'Revoir Paris',
                                   'type': 'article',
                                   'url': 'https://www.nytimes.com/2023/06/22/movies/revoir-paris-review-alice-winocour.html'},
                       'mpaa_rating': '',
       

In [ ]:
########################################################
########################################################
# NYT MOVIES API endpoint: best-sellers/history 
#-TEST ALL REQUEST
# The API sends back 500 articles meta for each section
# It is not possible to get more than 25000 articles
########################################################
########################################################

from elasticsearch import Elasticsearch
from pprint import pprint


# Create a connection to Elasticsearch  
#connect to ES container in local 
index_name = 'nyt-movies'
es = Elasticsearch(hosts = "http://@localhost:9200")


# Define the search query
search_query = {
    "query": {
        "match_all": {}
    }
}

# Perform the search request
response = es.search(index=index_name, body=search_query)

# Process and print the search results
pprint(response['hits']['hits'])

C:\Users\ep211.000\AppData\Local\Temp\ipykernel_13360\1139449770.py:28: DeprecationWarning: The 'body' parameter is deprecated and will be removed in a future version. Instead use individual parameters.
  response = es.search(index=index_name, body=search_query)


[{'_id': 'fERU4ogBhcLL25HdFYxP',
  '_index': 'nyt-movies',
  '_score': 1.0,
  '_source': {'byline': 'Glenn Kenny',
              'critics_pick': 0,
              'date_updated': '2023-06-21 16:26:03',
              'display_title': 'No Hard Feelings',
              'headline': '‘No Hard Feelings’ Review: How Lucky Can a Nerdy '
                          'Kid Get?',
              'link': {'suggested_link_text': 'Read the New York Times Review '
                                              'of No Hard Feelings',
                       'type': 'article',
                       'url': 'https://www.nytimes.com/2023/06/21/movies/no-hard-feelings-review.html'},
              'mpaa_rating': 'R',
              'multimedia': {'height': 140,
                             'src': 'https://static01.nyt.com/images/2023/06/22/multimedia/22no-hard-feelings-cvlh/22no-hard-feelings-cvlh-mediumThreeByTwo440.jpg',
                             'type': 'mediumThreeByTwo210',
                             'wid